Assignments:

To close out this checkpoint, you're going to do three assignments. For the first assignment, you'll write up a short answer to a question. For the second two assignments, you'll do your work in Jupyter notebooks.

Please submit links to all your work below. This is not a graded checkpoint, but you should discuss your solutions with your mentor. Also, when you're done, compare your work to these example solutions.

1.Interpretation and significance:

Suppose that we would like to know how much families in the US are spending on recreation annually. We've estimated the following model:

𝑒𝑥𝑝𝑒𝑛𝑑𝑖𝑡𝑢𝑟𝑒 = 873 + 0.0012 𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒 + 0.00002 𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒^2 − 223.57 ℎ𝑎𝑣𝑒_𝑘𝑖𝑑𝑠

expenditure is the annual spending on recreation in US dollars, annual_income is the annual income in US dollars, and have_kids is a dummy variable indicating the families with children. Interpret the estimated coefficients. What additional statistics should be given in order to make sure that your interpretations make sense statistically. Write up your answer.

For interpretating this model we should have t-statistics or the associated p-values. If the coefficants are not statistically significant we should consider them as a zero and interpretant the model in different way. 

I assume that all coefficients are statistically significant and their p_value are less than 0.1.

So, constants in linear regression models correspond to the outcome value that happen when all the explanatory variables take a zero value. The constant terms make the residuals zero on average. The constant is also called bias. So, in this model, the bias term is 873. 

For dummies variable: Have_kid: On average, families with kids spending 223.57 US dollars less than families without kids on annual expenditure of recreation.

For continuse variable: As you can see, the relationship between annual_income and expenditure is quadratic. So, if annual_income increase 1000 the expenditure increase $1.2 plus 0.02 * annual_income or 2% of annual_income. 


2.Weather model:

In this exercise, you'll work with the historical temperature data from the previous checkpoint. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

- First, load the dataset from the weatherinszeged table from Thinkful's database.

In [1]:
# Import some libraries and packages:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import bartlett
from scipy.stats import levene
from statsmodels.tsa.stattools import acf
from scipy.stats import jarque_bera
from scipy.stats import normaltest
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings(action="ignore")
%matplotlib inline
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Connect to dataset:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format( postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df = pd.read_sql_query('select * from weatherinszeged', con = engine)
engine.dispose()

In [3]:
# Look at the data set:
df.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.47,7.39,0.89,14.12,251.00,15.83,0.00,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.36,7.23,0.86,14.26,259.00,15.83,0.00,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.38,9.38,0.89,3.93,204.00,14.96,0.00,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.29,5.94,0.83,14.10,269.00,15.83,0.00,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.76,6.98,0.83,11.04,259.00,15.83,0.00,1016.51,Partly cloudy throughout the day.


- Build a linear regression model where your target variable is the difference between the apparenttemperature and the temperature. As explanatory variables, use humidity and windspeed.
- Now, estimate your model using OLS. Are the estimated coefficients statistically significant? 
- Are the signs of the estimated coefficients in line with your previous expectations? 
- Interpret the estimated coefficients.
- What are the relations between the target and the explanatory variables?

In [4]:
# Build a regression model:
# Y is the target variable:
Y = (df['apparenttemperature']-df['temperature'])
# X is the feature set which includes humidity, windspeed:
X = df[['humidity', 'windspeed']]
# We need to manually add a constant in statsmodels' sm
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Wed, 15 Jan 2020   Prob (F-statistic):               0.00
Time:                        23:33:10   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.000       2.397       2.479
humidity      -3.0292      0.024   -126.479      0.000      -3.076      -2.982
windspeed     -0.1193      0.001   -176.164      0.000      -0.121      -0.118
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                          -0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

- the linear regression model: 

Y = (df['apparenttemperature']-df['temperature']) = 2.44 - 3.03 humidity - 0.12 windspeed 

- As you can see p_values for coefficients are less than 0.1 or 0.05. So, the estimated coefficients statistically significant. 
- We expect the difference between the apparenttemperature and the temperature increase by increasing humidity and decrease by increasing windspeed. So, the sign of the estimated coefficient for humidity is not as our expected and it is negative that means by increasing humidity the diferences will decreased. But, the sign of the estimated coefficient for windspeed is as our expected and they have negatively related which means by increasing windspeed the differences will decreased.
- By increasing one point of humidity the difference between the apparenttemperature and the temperature will decreased by 3.03 point and by increasing one point of windspeed diffrences will decreased by 0.12 point.
- As you can see the relations between the target and the explanatory variables is negatively wich means by increasing explanatory variables the differences will decreased.

Next, include the interaction of humidity and windspeed to the model above and estimate the model using OLS. 
- Are the coefficients statistically significant? 
- Did the signs of the estimated coefficients for humidity and windspeed change? 
- Interpret the estimated coefficients.

In [5]:
# Y is the target variable
Y = (df['apparenttemperature']-df['temperature'])

# This is the interaction between humidity and windspeed:
df["humidity_windspeed"] = df.humidity * df.windspeed

# X is the feature set
X = df[['humidity', 'windspeed', 'humidity_windspeed']]

# We add a constant to the model as it's a best practice
# to do so every time!
X = sm.add_constant(X)

# We fit an OLS model using statsmodels
results = sm.OLS(Y, X).fit()

# We print the summary results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Wed, 15 Jan 2020   Prob (F-statistic):               0.00
Time:                        23:33:10   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0839      0

- the linear regression model: 

Y = (df['apparenttemperature']-df['temperature']) = 0.08 + 0.18 humidity + 0.09 windspeed - 0.30 humidity_windspeed ????correct???

- As you can see, the estimated coefficients statistically significant, because p_values for coefficients are less than 0.05. 
- Yes, by including the interaction of humidity and windspeed to the model, the sign of both explanatory variables are changed from negative to positive. 
- By increasing one point of humidity the difference between the apparenttemperature and the temperature will increased by 0.18 point and by increasing one point of windspeed, diffrences will increased by 0.09 point. About interaction variable, by increasing one point of humidity, target will increased 0.18-(0.30*windspeed) and by increasing one point in windspeed target will increased 0.09-(0.30*humidity). ???? correct?????

3. House prices model:

In this exercise, you'll interpret your house prices model. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:
- Load the houseprices data from Thinkful's database.

In [6]:
# Connect to the data base:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format( postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df = pd.read_sql_query('select * from houseprices', con=engine)
engine.dispose()

In [7]:
# Look at the dataset:
df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.00,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.00,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.00,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.00,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.00,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


- Run your house prices model again and interpret the results. Which features are statistically significant, and which are not?

In [8]:
# Convert categorical columns to numeric by using dummies:
categorical = df.select_dtypes(include=['object'])
dummies = pd.get_dummies(categorical, drop_first=True)
df = df.drop(categorical, axis=1)
df = pd.concat([df,dummies], axis=1)

In [10]:
# Now we build a simple linear regression model and estimate it using OLS:

# Y is the target variable:
Y = df['saleprice']
# X is the feature set:
X = df[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalbsmtsf', 'exterqual_TA', 'kitchenqual_TA']]

# We create a LinearRegression model with statsmodels' sm linear_model module.

# We need to manually add a constant in statsmodels' sm
X = sm.add_constant(X)

# fit method estimates the coefficients using OLS
results = sm.OLS(Y,  X).fit()

# Inspect the results.
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.767
Method:                 Least Squares   F-statistic:                     686.7
Date:                Wed, 15 Jan 2020   Prob (F-statistic):               0.00
Time:                        23:33:22   Log-Likelihood:                -17477.
No. Observations:                1460   AIC:                         3.497e+04
Df Residuals:                    1452   BIC:                         3.501e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const          -6.278e+04   7313.170     -8.584      0.000   -7.71e+04   -4.84e+04
overallqual     2.001e+04   1199.489     16.680      0.000    1.77e+04    2.24e+04
grlivarea         45.4666      2.457     18.505      0.000      40.647      50.286
garagecars        1.3e+04   2992.143      4.344      0.000    7129.524    1.89e+04
garagearea        16.9487     10.323      1.642      0.101      -3.301      37.199
totalbsmtsf       31.0423      2.868     10.825      0.000      25.417      36.667
exterqual_TA   -9898.1910   3146.166     -3.146      0.002   -1.61e+04   -3726.675
kitchenqual_TA -9741.9446   2776.148     -3.509      0.000   -1.52e+04   -4296.256
==============================================================================
Omnibus:                      429.417   Durbin-Watson:                   1.963
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            43903.389
Skew:                          -0.225   Prob(JB):                         0.00
Kurtosis:                      29.861   Cond. No.                     1.51e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.51e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Now, modeling the saleprice houses with linear regression is:

saleprice:y = -62780 + 20010 overallqual + 45 grlivarea + 13000 garagecars + 17 garagearea + 31 totalbsmtsf -9898 exterqual_TA - 9742 exterqual_TA + 𝜖

- All explanatory variables are  statistically significant except garagearea which is statistically insignificant, because its p_value is greater than 0.05. So its coefficient is statistically zero.
- By increasing one point of overallqual, grlivarea, garagecars, garagearea, totalbsmtsf, exterqual_TA and exterqual_TA, saleprice will be increased 20010, 45, 13000, 17 , 31 and decreased 9898 and 9742 respectively.

- Now, exclude the insignificant features from your model. Did anything change?

In [11]:
# Now we build a simple linear regression model and estimate it using OLS:

# Y is the target variable:
Y = df['saleprice']
# X is the feature set:
X = df[['overallqual', 'grlivarea', 'garagecars', 'totalbsmtsf', 'exterqual_TA', 'kitchenqual_TA']]

# We create a LinearRegression model with statsmodels' sm linear_model module.

# We need to manually add a constant in statsmodels' sm
X = sm.add_constant(X)

# fit method estimates the coefficients using OLS
results = sm.OLS(Y,  X).fit()

# Inspect the results.
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.767
Method:                 Least Squares   F-statistic:                     799.8
Date:                Wed, 15 Jan 2020   Prob (F-statistic):               0.00
Time:                        23:45:37   Log-Likelihood:                -17479.
No. Observations:                1460   AIC:                         3.497e+04
Df Residuals:                    1453   BIC:                         3.501e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const          -6.294e+04   7316.734     -8.603      0.000   -7.73e+04   -4.86e+04
overallqual     1.994e+04   1199.571     16.626      0.000    1.76e+04    2.23e+04
grlivarea         45.7596      2.452     18.663      0.000      40.950      50.569
garagecars      1.699e+04   1742.819      9.750      0.000    1.36e+04    2.04e+04
totalbsmtsf       32.0513      2.803     11.436      0.000      26.554      37.549
exterqual_TA   -9874.5415   3147.968     -3.137      0.002    -1.6e+04   -3699.494
kitchenqual_TA -9774.3667   2777.697     -3.519      0.000   -1.52e+04   -4325.643
==============================================================================
Omnibus:                      416.175   Durbin-Watson:                   1.962
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            39826.044
Skew:                          -0.160   Prob(JB):                         0.00
Kurtosis:                      28.585   Cond. No.                     1.47e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.47e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

saleprice:y = -62940 + 19940 overallqual + 46 grlivarea + 16990 garagecars + 32 totalbsmtsf -9875 exterqual_TA - 9774 exterqual_TA + 𝜖
- As you can see,the variables have changed a little.???? correct??

- Interpret the statistically significant coefficients by quantifying their relations with the house prices. Which features have a more prominent effect on house prices?

- As you can see, all coefficients are statistically significant because all p_values are less than 0.05.
- By increasing one point of overallqual, grlivarea, garagecars, garagearea, totalbsmtsf, exterqual_TA and exterqual_TA, saleprice will be increased 19940, 46, 16990, 32 and decreased 9875 and 9774 respectively..
- The largest coefficient is for overallqual which has a more prominent effect on house prices.?????correct???

- Do the results sound reasonable to you? If not, try to explain the potential reasons.

I think house saleprice depends on every thing such as location, overall quality, size, month of sale, zoning, etc. I investigated some high corelation features with saleprice,but,there are lots of other high correlated fetures with saleprice which is need to investigate, so it makes sense the result didnt seem reasonable much. ??????????correct??????????